# Логистическая регрессия, гиперпараметры, веса

## Данные
Как обычно, сначала загрузим данные и посмотрим на них.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('news_lenta_cropped.csv')

In [3]:
df.head()

,title,topic
0,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура
1,Попытка вернуть укатившийся мяч у Кремлевской ...,Силовые структуры
2,Хабенский стал врагом Украины,Культура
3,В Туркмении запретили продажу алкоголя,Бывший СССР
4,В Великобритании нашли утерянный сценарий Стен...,Культура


In [4]:
len(df)

129930

### Задание 1

Сколько всего значений принимает `topic`? Сколько объектов относится к каждому из топиков?

## Baseline

Посмотрим, как с классификацией справится наивный байес и CountVectorizer с дефолтными настройками.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [6]:
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(df.title)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(bow, df.topic)

In [28]:
nb = MultinomialNB()
clf = nb.fit(X_train, y_train)

In [29]:
from sklearn.metrics import classification_report

In [30]:
print(classification_report(y_test, clf.predict(X_test)))

                   precision    recall  f1-score   support

      Бывший СССР       0.92      0.97      0.95     13004
         Культура       0.96      0.94      0.95     13176
Силовые структуры       0.90      0.85      0.88      4636
         Ценности       0.93      0.78      0.85      1667

      avg / total       0.93      0.93      0.93     32483



## LogisticRegression

In [11]:
from sklearn.linear_model import LogisticRegression

In [41]:
lr = LogisticRegression(random_state=42)
clf = lr.fit(X_train, y_train)

In [42]:
print(classification_report(y_test, clf.predict(X_test)))

                   precision    recall  f1-score   support

      Бывший СССР       0.96      0.97      0.96     13004
         Культура       0.94      0.97      0.95     13176
Силовые структуры       0.93      0.87      0.90      4636
         Ценности       0.95      0.77      0.85      1667

      avg / total       0.94      0.94      0.94     32483



Получилось чууть получше. Но что если?..

## Подбор параметров

### Вручную

In [40]:
lr = LogisticRegression(C=2, random_state=42)
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

                   precision    recall  f1-score   support

      Бывший СССР       0.96      0.97      0.96     13004
         Культура       0.94      0.97      0.96     13176
Силовые структуры       0.93      0.88      0.90      4636
         Ценности       0.94      0.80      0.86      1667

      avg / total       0.95      0.95      0.95     32483



Ю-ху, ещё лучше!

### В цикле

### Grid Search

Самый продвинутый и out-of-box способ делать это.

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
clf = LogisticRegression()
grid_values = {'penalty': ['l1', 'l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train, y_train)

## SGD

А теперь попробуйте проделать то же самое с `SGDClassifier`.

In [14]:
from sklearn.linear_model import SGDClassifier

In [43]:
sgd = SGDClassifier()
# your code here